# In-Class Assignment — Data Preprocessing & Cleaning (Text)  
**Time:** 20 minutes  |  **Points:** 10  

## Instructions
- This is an individual in-class assignment.  
- Write your code **inside each answer cell**.  
- Print the required outputs.  
- Submit your GitHub/Colab link as instructed by the instructor.


You are given a small dataset of customer support messages as a **TAB-separated text file**:  
- `support_messages.txt`

You will download this file from **Canvas** and upload it to your **Google Colab** notebook.

**How to upload it to your Google Colab notebook?**

1. Download `support_messages.txt` from Canvas.
3. In **the left sidebar**, click the **Files** icon (folder).  
4. Click **Upload** and select `support_messages.txt`.

6. RightAfter uploading, the file will appear in the Colab file list on the left.

6. Right-click the file, copy its path, and paste it into the FILE_PATH variable in Q1.

7. Run Q1 to load the dataset.



> Important: Keep the file name exactly as `support_messages.txt`.


## Questions (Total = 10 points)

### Q1 (1 point) — Load the dataset
Load the TAB-separated file into a pandas DataFrame with columns: `id`, `message`.  
Print: **(a)** `df.shape`, **(b)** `df.head(3)`.

### Q2 (3 points) — Descriptive columns
Add these columns for each message and print the full DataFrame:
- `word_count`: number of words  
- `char_count`: number of characters  
- `num_count`: number of digits (0–9)  
- `upper_word_count`: number of ALL-CAPS words (e.g., `"WHY"`, `"DAMAGED"`)  

### Q3 (3 points) — Clean text
Build a `clean_text(text)` function and create a new column `clean` with these steps **in order**:
1) lowercase  
2) remove punctuation/symbols (keep letters/numbers/spaces)  
3) remove English stopwords (use **nltk** or **sklearn** list)  
4) remove extra spaces  

Print the **original** message and **clean** version for rows `id=1` and `id=4`.

### Q4 (2 points) — Regex extraction
Using RegEx, extract and create two new columns:
- `order_id`: first occurrence of pattern `ORD-####` (case-insensitive; `ord-1060` is valid)  
- `email`: first email address if present (otherwise `None`/`NaN`)  

Print: `id`, `order_id`, `email` for all rows.

### Q5 (1 point) — TF-IDF keywords
Using the `clean` column, compute **TF-IDF** for the messages and print the **top 5 keywords** with the highest **average TF-IDF** across documents.


In [ ]:
# Setup (run this cell first)
import re
import pandas as pd


## Q1 (1 point) — Answer below

In [12]:
import pandas as pd

FILE_PATH = "/content/support_messages.txt"  # Corrected path for Google Colab

# Load TAB-separated file
df = pd.read_csv(FILE_PATH, sep="\t", names=["id", "message"])

# Print required outputs
print(df.shape)
print(df.head(3))

(9, 2)
   id                                            message
0  id                                            message
1   1  Hi!! My ORDER is late :(  Order# ORD-1042. Ema...
2   2  Refund please!!! I was charged 2 times... invo...


## Q2 (3 points) — Answer below

In [14]:
# Q2 — ANSWER CELL

# word count
df["word_count"] = df["message"].astype(str).apply(lambda x: len(x.split()))

# character count
df["char_count"] = df["message"].astype(str).apply(len)

# digit count
df["num_count"] = df["message"].astype(str).str.count(r"\d")

# ALL-CAPS word count
def count_upper_words(text):
    return sum(1 for word in str(text).split() if word.isupper())

df["upper_word_count"] = df["message"].apply(count_upper_words)

# Display full DataFrame
print(df)


   id                                            message  word_count  \
0  id                                            message           1   
1   1  Hi!! My ORDER is late :(  Order# ORD-1042. Ema...          12   
2   2  Refund please!!! I was charged 2 times... invo...          11   
3   3        Great service, thanks! arrived in 2 days :)           8   
4   4  WHY is my package DAMAGED??? tracking says del...           8   
5   5  Need to change address: 7421 Frankford Rd Apt ...          12   
6   6  Support ticket: ORD-1050. Call me at (469) 555...           9   
7   7  I can’t login— password reset link not working...          10   
8   8  Item missing from box. pls send replacement!! ...          10   

   char_count  num_count  upper_word_count  
0           7          0                 0  
1          73          4                 2  
2          71          9                 3  
3          43          1                 0  
4          55          0                 2  
5        

## Q3 (3 points) — Answer below

In [36]:
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

STOPWORDS = set(ENGLISH_STOP_WORDS)

def clean_text(text):
    # convert to string (prevents errors)
    text = str(text).lower()

    # remove punctuation/symbols
    text = re.sub(r"[^a-z0-9\s]", " ", text)

    # remove stopwords
    tokens = [word for word in text.split() if word not in STOPWORDS]

    # remove extra spaces
    return " ".join(tokens)

# Create clean column
df["clean"] = df["message"].apply(clean_text)

# Print original and clean for id = 1 and id = 4
print(df[df["id"].astype(str).isin(['1', '4'])][["id", "message", "clean"]])

  id                                            message  \
1  1  Hi!! My ORDER is late :(  Order# ORD-1042. Ema...   
4  4  WHY is my package DAMAGED??? tracking says del...   

                                               clean  
1  hi order late order ord 1042 email sara ali gm...  
4            package damaged tracking says delivered  


## Q4 (2 points) — Answer below

In [22]:
import re

# Extract order ID
df["order_id"] = df["message"].str.extract(
    r"(ORD-\d{4})", flags=re.IGNORECASE
)

# Extract email
df["email"] = df["message"].str.extract(
    r"([A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,})"
)

# Print required columns
print(df[["id", "order_id", "email"]])

   id  order_id                  email
0  id       NaN                    NaN
1   1  ORD-1042     sara.Ali@gmail.com
2   2  ORD-1042                    NaN
3   3       NaN                    NaN
4   4       NaN                    NaN
5   5       NaN                    NaN
6   6  ORD-1050                    NaN
7   7       NaN  mehri.sattari@unt.edu
8   8  ord-1060                    NaN


## Q5 (1 point) — Answer below

In [25]:
# SAFE CELL: creates clean column if it does not exist

import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Check columns
print("Before:", df.columns.tolist())

STOPWORDS = set(ENGLISH_STOP_WORDS)

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = " ".join([w for w in text.split() if w not in STOPWORDS])
    return text

# Create clean column ONLY if missing
if "clean" not in df.columns:
    df["clean"] = df["message"].apply(clean_text)

print("After:", df.columns.tolist())

Before: ['id', 'message', 'word_count', 'char_count', 'num_count', 'upper_word_count', 'order_id', 'email']
After: ['id', 'message', 'word_count', 'char_count', 'num_count', 'upper_word_count', 'order_id', 'email', 'clean']


## Grading Checklist
- Q1: correct load + prints  
- Q2: correct counts  
- Q3: cleaning follows the required order + prints for id=1 and id=4  
- Q4: regex extraction works (case-insensitive `ORD-####` and emails)  
- Q5: prints 5 keywords + their scores (rounding is fine)
